# Module 09: Probability and Risk Management

**Difficulty**: ⭐⭐ Intermediate  
**Estimated Time**: 60 minutes  
**Prerequisites**: 
- Module 02: Spread and Variation (standard deviation, variance)
- Module 03: Percentages, Ratios, and Changes
- Module 08: Correlation and Relationships
- Basic probability concepts

## Learning Objectives

By the end of this notebook, you will be able to:
1. **Calculate expected value** of a trading strategy mathematically
2. **Understand win rate vs risk/reward ratio** tradeoffs
3. **Calculate position sizing** using fixed fraction and Kelly Criterion
4. **Measure maximum drawdown** and understand its implications
5. **Calculate risk of ruin** probability
6. **Apply risk management formulas** to real trading scenarios

---

## Why Probability and Risk Management Matter

You can have the **best technical analysis** in the world, but without proper risk management, you'll still lose money!

### The Hard Truth:

- **Most traders lose money** - not because they can't predict, but because they can't manage risk
- **One big loss** can wipe out months of small wins
- **Position sizing** matters more than win rate
- **Probability mathematics** gives you an edge

### What We'll Learn:

This module teaches you the **mathematics of survival** in trading:
- How much to risk per trade (position sizing)
- How to calculate if a strategy is profitable long-term (expected value)
- How to avoid blowing up your account (risk of ruin)

Let's dive into the math that separates winners from losers!

---

In [ ]:
# Setup and imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
from scipy import stats
import warnings

# Configuration
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
warnings.filterwarnings('ignore')
np.random.seed(42)

# Plot settings
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("✓ Libraries imported successfully")
print(f"Today's date: {datetime.now().strftime('%Y-%m-%d')}")

## 1. Expected Value: The Foundation of Profitable Trading

**Expected Value (EV)** tells you the **average profit or loss** per trade over many trades.

### The Formula:

$$EV = (P_{\text{win}} \times \text{Avg Win}) - (P_{\text{loss}} \times \text{Avg Loss})$$

Where:
- $P_{\text{win}}$ = Probability of winning (win rate)
- $P_{\text{loss}}$ = Probability of losing = 1 - $P_{\text{win}}$
- Avg Win = Average profit when you win
- Avg Loss = Average loss when you lose

### Interpretation:

| Expected Value | Meaning | Action |
|----------------|---------|--------|
| **EV > 0** | Profitable strategy | Trade it! |
| **EV = 0** | Break-even | Don't waste time |
| **EV < 0** | Losing strategy | Don't trade! |

### Example 1: High Win Rate, Small Wins

- Win rate: 70%
- Average win: RM 100
- Average loss: RM 300

$$EV = (0.70 \times 100) - (0.30 \times 300) = 70 - 90 = -20$$

**Result**: Negative EV! Even with 70% win rate, you lose money!

### Example 2: Low Win Rate, Big Wins

- Win rate: 30%
- Average win: RM 500
- Average loss: RM 100

$$EV = (0.30 \times 500) - (0.70 \times 100) = 150 - 70 = +80$$

**Result**: Positive EV! Only 30% win rate but still profitable!

### The Key Insight:

**Win rate alone doesn't matter - it's the combination of win rate and risk/reward ratio!**

---

In [ ]:
# Calculate expected value
def calculate_expected_value(win_rate, avg_win, avg_loss):
    """
    Calculate expected value of a trading strategy.
    
    Parameters:
    -----------
    win_rate : float
        Probability of winning (0 to 1)
    avg_win : float
        Average profit when winning
    avg_loss : float
        Average loss when losing (positive number)
    
    Returns:
    --------
    dict : Expected value and related metrics
    """
    loss_rate = 1 - win_rate
    
    # Expected value formula
    ev = (win_rate * avg_win) - (loss_rate * avg_loss)
    
    # Risk/Reward ratio
    risk_reward_ratio = avg_win / avg_loss if avg_loss > 0 else 0
    
    # Expectancy (EV per dollar risked)
    expectancy = ev / avg_loss if avg_loss > 0 else 0
    
    return {
        'Expected Value': ev,
        'Win Rate': win_rate,
        'Loss Rate': loss_rate,
        'Avg Win': avg_win,
        'Avg Loss': avg_loss,
        'Risk/Reward Ratio': risk_reward_ratio,
        'Expectancy': expectancy,
        'Profitable': 'YES' if ev > 0 else 'NO'
    }


# Test different scenarios
scenarios = [
    {'name': 'High Win Rate, Small Wins', 'win_rate': 0.70, 'avg_win': 100, 'avg_loss': 300},
    {'name': 'Low Win Rate, Big Wins', 'win_rate': 0.30, 'avg_win': 500, 'avg_loss': 100},
    {'name': 'Balanced Strategy', 'win_rate': 0.50, 'avg_win': 200, 'avg_loss': 150},
    {'name': 'Poor Strategy', 'win_rate': 0.40, 'avg_win': 100, 'avg_loss': 150},
]

print("Expected Value Analysis for Different Trading Strategies:")
print("=" * 90)

results = []
for scenario in scenarios:
    result = calculate_expected_value(
        scenario['win_rate'], 
        scenario['avg_win'], 
        scenario['avg_loss']
    )
    result['Strategy'] = scenario['name']
    results.append(result)
    
    print(f"\n📊 {scenario['name']}:")
    print("-" * 90)
    for key, value in result.items():
        if isinstance(value, float):
            print(f"  {key:.<45} {value:.2f}")
        else:
            print(f"  {key:.<45} {value}")

# Create DataFrame for visualization
results_df = pd.DataFrame(results)

print("\n" + "=" * 90)
print("💡 Key Insight: Win rate alone doesn't guarantee profitability!")
print("   Focus on the COMBINATION of win rate and risk/reward ratio.")

In [ ]:
# Visualize expected value for different strategies
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left panel: Expected value comparison
colors = ['green' if ev > 0 else 'red' for ev in results_df['Expected Value']]
axes[0].barh(results_df['Strategy'], results_df['Expected Value'], color=colors, alpha=0.7)
axes[0].axvline(x=0, color='black', linestyle='--', linewidth=2)
axes[0].set_xlabel('Expected Value (RM per trade)', fontsize=11)
axes[0].set_title('Expected Value Comparison', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='x')

# Add value labels
for i, v in enumerate(results_df['Expected Value']):
    axes[0].text(v, i, f' RM {v:.2f}', va='center', fontsize=9, fontweight='bold')

# Right panel: Win Rate vs Risk/Reward Ratio scatter
scatter = axes[1].scatter(results_df['Win Rate'] * 100, 
                          results_df['Risk/Reward Ratio'],
                          s=200, c=results_df['Expected Value'], 
                          cmap='RdYlGn', alpha=0.7,
                          edgecolors='black', linewidths=2)

# Add labels for each point
for idx, row in results_df.iterrows():
    axes[1].annotate(row['Strategy'], 
                     (row['Win Rate'] * 100, row['Risk/Reward Ratio']),
                     fontsize=8, ha='right', va='bottom')

axes[1].set_xlabel('Win Rate (%)', fontsize=11)
axes[1].set_ylabel('Risk/Reward Ratio', fontsize=11)
axes[1].set_title('Win Rate vs Risk/Reward (Color = EV)', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Add colorbar
cbar = plt.colorbar(scatter, ax=axes[1])
cbar.set_label('Expected Value', fontsize=10)

plt.tight_layout()
plt.show()

print("\n📊 Chart Interpretation:")
print("=" * 70)
print("Left: Green bars = profitable strategies (EV > 0)")
print("      Red bars = losing strategies (EV < 0)")
print("\nRight: Greener points = higher EV")
print("       Notice: Low win rate can still be profitable with high R/R!")

## 2. Position Sizing: How Much to Risk

**Position sizing** is arguably the **most important** trading decision.

### Why Position Sizing Matters:

- Risk **too much**: One bad streak wipes you out
- Risk **too little**: Takes forever to grow capital
- Risk **optimal amount**: Maximize growth while controlling risk

### Common Position Sizing Methods:

#### 1. Fixed Dollar Amount
- Risk the same RM amount per trade (e.g., RM 100)
- **Simple** but doesn't scale with account size

#### 2. Fixed Percentage (Fixed Fractional)
- Risk a fixed percentage of capital (e.g., 2%)
- **Most common** among professional traders
- **Formula**: 

$$\text{Position Size} = \frac{\text{Account Size} \times \text{Risk \%}}{\text{Stop Loss Distance (in RM)}}$$

#### 3. Kelly Criterion
- Optimal bet size to **maximize long-term growth**
- **Formula**: 

$$f^* = \frac{p}{a} - \frac{q}{b}$$

Where:
- $f^*$ = Fraction of capital to risk
- $p$ = Probability of winning
- $q$ = Probability of losing = 1 - p
- $a$ = Amount lost per losing trade (as decimal, e.g., 0.10 for 10% loss)
- $b$ = Amount won per winning trade (as decimal)

**Simplified Kelly** (for trading):

$$\text{Kelly \%} = \text{Win Rate} - \frac{\text{Loss Rate}}{\text{Win/Loss Ratio}}$$

### ⚠️ Important:

- Most traders use **half-Kelly** or **quarter-Kelly** (less aggressive)
- Full Kelly is **too aggressive** for most traders
- Never risk more than 5% per trade (even if Kelly says so)

Let's calculate position sizes for real scenarios!

---

In [ ]:
# Fixed percentage position sizing
def calculate_position_size_fixed_pct(account_size, risk_pct, entry_price, stop_loss_price):
    """
    Calculate position size using fixed percentage method.
    
    Parameters:
    -----------
    account_size : float
        Total account size in RM
    risk_pct : float
        Percentage of account to risk per trade (e.g., 0.02 for 2%)
    entry_price : float
        Entry price per share
    stop_loss_price : float
        Stop loss price per share
    
    Returns:
    --------
    dict : Position sizing details
    """
    # Calculate risk per share
    risk_per_share = abs(entry_price - stop_loss_price)
    
    # Calculate total risk amount
    risk_amount = account_size * risk_pct
    
    # Calculate number of shares
    num_shares = risk_amount / risk_per_share
    
    # Calculate position value
    position_value = num_shares * entry_price
    
    # Calculate position as % of account
    position_pct = (position_value / account_size) * 100
    
    return {
        'Account Size': account_size,
        'Risk %': risk_pct * 100,
        'Risk Amount': risk_amount,
        'Entry Price': entry_price,
        'Stop Loss': stop_loss_price,
        'Risk per Share': risk_per_share,
        'Number of Shares': int(num_shares),
        'Position Value': position_value,
        'Position % of Account': position_pct
    }


# Example: Trading Maybank
print("Fixed Percentage Position Sizing Example:")
print("=" * 80)
print("\nScenario: Trading Maybank (1155.KL)")
print("  Account size: RM 50,000")
print("  Entry price: RM 9.00")
print("  Stop loss: RM 8.50 (5.6% below entry)")
print("  Risk per trade: 2% of account")
print()

position = calculate_position_size_fixed_pct(
    account_size=50000,
    risk_pct=0.02,
    entry_price=9.00,
    stop_loss_price=8.50
)

print("Position Sizing Calculation:")
print("-" * 80)
for key, value in position.items():
    if isinstance(value, float):
        if 'RM' in key or key in ['Risk Amount', 'Position Value', 'Account Size']:
            print(f"{key:.<50} RM {value:,.2f}")
        elif '%' in key:
            print(f"{key:.<50} {value:.2f}%")
        else:
            print(f"{key:.<50} {value:.2f}")
    else:
        print(f"{key:.<50} {value:,}")

print("\n💡 Interpretation:")
print(f"  Buy {int(position['Number of Shares']):,} shares at RM {position['Entry Price']:.2f}")
print(f"  Total investment: RM {position['Position Value']:,.2f} ({position['Position % of Account']:.1f}% of account)")
print(f"  If stopped out: Lose RM {position['Risk Amount']:,.2f} (exactly 2% of account)")

In [ ]:
# Kelly Criterion position sizing
def calculate_kelly_criterion(win_rate, avg_win, avg_loss):
    """
    Calculate optimal position size using Kelly Criterion.
    
    Parameters:
    -----------
    win_rate : float
        Probability of winning (0 to 1)
    avg_win : float
        Average win amount (can be ratio or absolute)
    avg_loss : float
        Average loss amount (positive number)
    
    Returns:
    --------
    dict : Kelly percentage and recommendations
    """
    loss_rate = 1 - win_rate
    win_loss_ratio = avg_win / avg_loss
    
    # Kelly formula
    kelly_pct = win_rate - (loss_rate / win_loss_ratio)
    
    # Conservative variants
    half_kelly = kelly_pct / 2
    quarter_kelly = kelly_pct / 4
    
    return {
        'Full Kelly %': kelly_pct * 100,
        'Half Kelly %': half_kelly * 100,
        'Quarter Kelly %': quarter_kelly * 100,
        'Recommendation': 'Half Kelly' if kelly_pct > 0 else 'Do not trade',
        'Win Rate': win_rate * 100,
        'Win/Loss Ratio': win_loss_ratio
    }


# Calculate Kelly for our scenarios
print("Kelly Criterion Position Sizing:")
print("=" * 80)

for scenario in scenarios:
    kelly = calculate_kelly_criterion(
        scenario['win_rate'],
        scenario['avg_win'],
        scenario['avg_loss']
    )
    
    print(f"\n📊 {scenario['name']}:")
    print("-" * 80)
    for key, value in kelly.items():
        if isinstance(value, float):
            if '%' in key:
                print(f"  {key:.<50} {value:.2f}%")
            else:
                print(f"  {key:.<50} {value:.2f}")
        else:
            print(f"  {key:.<50} {value}")

print("\n" + "=" * 80)
print("⚠️  WARNING: Full Kelly is VERY aggressive!")
print("   Most professional traders use Half-Kelly or Quarter-Kelly.")
print("   Never exceed 5% risk per trade regardless of Kelly calculation.")

In [ ]:
# Simulate account growth with different position sizing
def simulate_trading(initial_capital, num_trades, win_rate, avg_win_pct, avg_loss_pct, 
                     position_size_pct, num_simulations=1000):
    """
    Monte Carlo simulation of trading with different position sizes.
    """
    results = []
    
    for _ in range(num_simulations):
        capital = initial_capital
        equity_curve = [capital]
        
        for trade in range(num_trades):
            # Random outcome based on win rate
            if np.random.random() < win_rate:
                # Win
                profit = capital * position_size_pct * avg_win_pct
            else:
                # Loss
                profit = -capital * position_size_pct * avg_loss_pct
            
            capital += profit
            equity_curve.append(capital)
            
            # Stop if ruined
            if capital <= 0:
                capital = 0
                break
        
        results.append({
            'Final Capital': capital,
            'Return %': ((capital - initial_capital) / initial_capital) * 100,
            'Equity Curve': equity_curve,
            'Ruined': capital <= 0
        })
    
    return results


# Run simulations
print("Monte Carlo Simulation: Position Sizing Impact")
print("=" * 80)
print("\nSimulation Parameters:")
print("  Initial capital: RM 50,000")
print("  Number of trades: 100")
print("  Win rate: 50%")
print("  Avg win: 4% per position")
print("  Avg loss: 2% per position")
print("  Number of simulations: 1000")
print()

# Test different position sizes
position_sizes = [0.25, 0.50, 1.00, 2.00]  # 25%, 50%, 100%, 200% of capital per trade
simulation_results = {}

for pos_size in position_sizes:
    results = simulate_trading(
        initial_capital=50000,
        num_trades=100,
        win_rate=0.50,
        avg_win_pct=0.04,
        avg_loss_pct=0.02,
        position_size_pct=pos_size,
        num_simulations=1000
    )
    simulation_results[pos_size] = results
    
    # Calculate statistics
    final_capitals = [r['Final Capital'] for r in results]
    returns = [r['Return %'] for r in results]
    ruin_rate = sum([r['Ruined'] for r in results]) / len(results) * 100
    
    print(f"Position Size: {pos_size * 100:.0f}% of capital per trade")
    print("-" * 80)
    print(f"  Median final capital: RM {np.median(final_capitals):,.2f}")
    print(f"  Mean return: {np.mean(returns):.2f}%")
    print(f"  Median return: {np.median(returns):.2f}%")
    print(f"  Risk of ruin: {ruin_rate:.2f}%")
    print()

print("💡 Key Insight: Smaller position sizes = Lower risk of ruin!")

In [ ]:
# Visualize simulation results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, pos_size in enumerate(position_sizes):
    results = simulation_results[pos_size]
    
    # Plot sample equity curves
    for i in range(min(50, len(results))):  # Plot first 50 simulations
        axes[idx].plot(results[i]['Equity Curve'], alpha=0.1, color='blue')
    
    # Plot median equity curve
    median_curve = np.median([r['Equity Curve'] for r in results if len(r['Equity Curve']) == 101], axis=0)
    axes[idx].plot(median_curve, color='red', linewidth=3, label='Median')
    
    # Horizontal line at initial capital
    axes[idx].axhline(y=50000, color='green', linestyle='--', linewidth=2, label='Break Even')
    axes[idx].axhline(y=0, color='black', linestyle='-', linewidth=1)
    
    # Calculate ruin rate
    ruin_rate = sum([r['Ruined'] for r in results]) / len(results) * 100
    
    axes[idx].set_title(f'Position Size: {pos_size * 100:.0f}% (Ruin Rate: {ruin_rate:.1f}%)', 
                        fontsize=11, fontweight='bold')
    axes[idx].set_xlabel('Trade Number', fontsize=10)
    axes[idx].set_ylabel('Account Value (RM)', fontsize=10)
    axes[idx].legend(loc='best', fontsize=9)
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_ylim(0, max(200000, np.max(median_curve)))

plt.tight_layout()
plt.show()

print("\n📊 Chart Interpretation:")
print("=" * 70)
print("Each panel shows 1000 simulations with different position sizes:")
print("  - Blue lines: Individual simulation paths")
print("  - Red line: Median outcome")
print("  - Green dashed: Break-even point")
print("\nNotice:")
print("  - Smaller position sizes → More stable growth, lower ruin risk")
print("  - Larger position sizes → Higher variance, higher ruin risk")
print("  - 200% position size → Many paths hit zero (ruined)!")

## 3. Maximum Drawdown: Measuring Pain

**Maximum Drawdown (MDD)** is the **largest peak-to-trough decline** in your account.

### Why MDD Matters:

1. **Psychological pain**: Large drawdowns are hard to stomach
2. **Recovery difficulty**: A 50% loss requires 100% gain to recover!
3. **Strategy assessment**: High MDD = high risk strategy

### The Mathematics:

$$\text{Drawdown}_t = \frac{\text{Equity}_t - \text{Peak Equity}_{t}}{\text{Peak Equity}_{t}} \times 100\%$$

$$\text{Maximum Drawdown} = \max(\text{Drawdown}_t) \text{ for all } t$$

### Recovery Mathematics:

To recover from a drawdown of $D\%$, you need a gain of:

$$\text{Recovery Gain} = \frac{D}{1 - D} \times 100\%$$

**Examples**:
- 10% drawdown → Need 11.1% gain to recover
- 25% drawdown → Need 33.3% gain to recover
- 50% drawdown → Need 100% gain to recover!
- 75% drawdown → Need 300% gain to recover!!!

### The Takeaway:

**Avoid large drawdowns at all costs!** They're extremely hard to recover from.

---

In [ ]:
# Calculate maximum drawdown
def calculate_max_drawdown(equity_curve):
    """
    Calculate maximum drawdown from an equity curve.
    
    Parameters:
    -----------
    equity_curve : array-like
        Series of account values over time
    
    Returns:
    --------
    dict : Drawdown statistics
    """
    equity = np.array(equity_curve)
    
    # Calculate running maximum (peak)
    running_max = np.maximum.accumulate(equity)
    
    # Calculate drawdown at each point
    drawdown = (equity - running_max) / running_max * 100
    
    # Find maximum drawdown
    max_dd = np.min(drawdown)
    max_dd_idx = np.argmin(drawdown)
    
    # Find the peak before max drawdown
    peak_idx = np.argmax(running_max[:max_dd_idx+1])
    peak_value = equity[peak_idx]
    trough_value = equity[max_dd_idx]
    
    # Calculate recovery gain needed
    recovery_gain = (abs(max_dd) / (100 - abs(max_dd))) * 100
    
    return {
        'Max Drawdown %': max_dd,
        'Peak Value': peak_value,
        'Trough Value': trough_value,
        'Peak Index': peak_idx,
        'Trough Index': max_dd_idx,
        'Recovery Gain Needed %': recovery_gain,
        'Drawdown Series': drawdown,
        'Running Max': running_max
    }


# Demonstrate recovery difficulty
print("Maximum Drawdown and Recovery Mathematics:")
print("=" * 80)

drawdown_levels = [10, 20, 30, 40, 50, 60, 70, 80, 90]

print("\nDrawdown vs Recovery Gain Needed:")
print("-" * 80)
print(f"{'Drawdown':<15} {'Starting $':<15} {'After Loss':<15} {'Gain Needed':<15} {'End $':<15}")
print("-" * 80)

initial = 10000
for dd in drawdown_levels:
    after_loss = initial * (1 - dd/100)
    recovery_pct = (dd / (100 - dd)) * 100
    end_value = after_loss * (1 + recovery_pct/100)
    
    print(f"{dd}%{'':<12} ${initial:,.0f}{'':<7} ${after_loss:,.0f}{'':<7} "
          f"{recovery_pct:>5.1f}%{'':<8} ${end_value:,.0f}")

print("\n💡 Key Insight:")
print("=" * 80)
print("A 50% loss requires a 100% gain just to break even!")
print("A 75% loss requires a 300% gain to recover!")
print("\n⚠️  AVOID LARGE DRAWDOWNS - They're almost impossible to recover from!")

In [ ]:
# Analyze drawdown from a sample equity curve
# Let's use one simulation from earlier
sample_result = simulation_results[0.50][0]  # 50% position size, first simulation
equity_curve = sample_result['Equity Curve']

dd_stats = calculate_max_drawdown(equity_curve)

print("Sample Trading Account Drawdown Analysis:")
print("=" * 80)
print(f"Peak value: RM {dd_stats['Peak Value']:,.2f} (at trade #{dd_stats['Peak Index']})")
print(f"Trough value: RM {dd_stats['Trough Value']:,.2f} (at trade #{dd_stats['Trough Index']})")
print(f"Maximum drawdown: {dd_stats['Max Drawdown %']:.2f}%")
print(f"Recovery gain needed: {dd_stats['Recovery Gain Needed %']:.2f}%")
print()

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Panel 1: Equity curve with peak and trough marked
axes[0].plot(equity_curve, color='blue', linewidth=2, label='Account Value')
axes[0].plot(dd_stats['Running Max'], color='green', linestyle='--', linewidth=2, label='Peak (Running Max)')
axes[0].scatter([dd_stats['Peak Index']], [dd_stats['Peak Value']], 
                color='green', s=200, zorder=5, marker='^', label='Peak')
axes[0].scatter([dd_stats['Trough Index']], [dd_stats['Trough Value']], 
                color='red', s=200, zorder=5, marker='v', label='Trough')
axes[0].fill_between(range(len(equity_curve)), equity_curve, dd_stats['Running Max'],
                      where=(np.array(equity_curve) < dd_stats['Running Max']),
                      color='red', alpha=0.3, label='Drawdown')
axes[0].set_ylabel('Account Value (RM)', fontsize=11)
axes[0].set_title('Equity Curve and Maximum Drawdown', fontsize=12, fontweight='bold')
axes[0].legend(loc='best')
axes[0].grid(True, alpha=0.3)

# Panel 2: Drawdown over time
axes[1].fill_between(range(len(dd_stats['Drawdown Series'])), 
                      dd_stats['Drawdown Series'], 0,
                      color='red', alpha=0.5)
axes[1].plot(dd_stats['Drawdown Series'], color='darkred', linewidth=2)
axes[1].axhline(y=dd_stats['Max Drawdown %'], color='black', 
                linestyle='--', linewidth=2, label=f'Max DD: {dd_stats["Max Drawdown %"]:.2f}%')
axes[1].set_ylabel('Drawdown (%)', fontsize=11)
axes[1].set_xlabel('Trade Number', fontsize=11)
axes[1].set_title('Drawdown Over Time', fontsize=12, fontweight='bold')
axes[1].legend(loc='best')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Chart Interpretation:")
print("=" * 70)
print("Top: Equity curve (blue) with running peak (green dashed)")
print("     Red shading shows periods of drawdown")
print("     Triangle markers show peak and trough of max drawdown")
print("\nBottom: Drawdown percentage over time")
print("        Lower = worse (further from zero)")

## 4. Risk of Ruin: Probability of Blowing Up

**Risk of Ruin** = Probability that you'll lose all your trading capital.

### The Formula (simplified):

For equal bet sizes:

$$R = \left(\frac{1-A}{1+A}\right)^{U}$$

Where:
- $R$ = Risk of ruin
- $A$ = Player advantage (your edge)
- $U$ = Number of units of capital

**Player Advantage**:

$$A = \frac{(\text{Win Rate} \times \text{Avg Win}) - (\text{Loss Rate} \times \text{Avg Loss})}{\text{Average Bet Size}}$$

### Key Insights:

1. **Negative edge → 100% ruin eventually**
2. **Smaller bet sizes → Lower risk of ruin**
3. **More capital units → Lower risk of ruin**

### Practical Rule of Thumb:

To keep risk of ruin < 1%:
- Have at least **50-100 units** of capital
- If you risk 2% per trade → 50 units = Can survive 50 losing trades
- If you risk 1% per trade → 100 units = Can survive 100 losing trades

---

In [ ]:
# Calculate risk of ruin
def calculate_risk_of_ruin(win_rate, avg_win, avg_loss, risk_per_trade_pct):
    """
    Calculate risk of ruin for a trading strategy.
    
    Parameters:
    -----------
    win_rate : float
        Win rate (0 to 1)
    avg_win : float
        Average win as percentage (e.g., 0.05 for 5%)
    avg_loss : float
        Average loss as percentage (e.g., 0.03 for 3%)
    risk_per_trade_pct : float
        Risk per trade as percentage (e.g., 0.02 for 2%)
    
    Returns:
    --------
    dict : Risk of ruin and related metrics
    """
    loss_rate = 1 - win_rate
    
    # Calculate player advantage
    expected_return = (win_rate * avg_win) - (loss_rate * avg_loss)
    avg_bet = (avg_win + avg_loss) / 2
    advantage = expected_return / avg_bet if avg_bet > 0 else 0
    
    # Number of units
    units = 1 / risk_per_trade_pct
    
    # Risk of ruin formula
    if advantage <= 0:
        risk_of_ruin = 1.0  # Certain ruin with negative edge
    else:
        risk_of_ruin = ((1 - advantage) / (1 + advantage)) ** units
    
    return {
        'Win Rate': win_rate * 100,
        'Avg Win %': avg_win * 100,
        'Avg Loss %': avg_loss * 100,
        'Expected Return %': expected_return * 100,
        'Player Advantage': advantage,
        'Risk per Trade %': risk_per_trade_pct * 100,
        'Capital Units': units,
        'Risk of Ruin %': risk_of_ruin * 100
    }


# Test different scenarios
print("Risk of Ruin Analysis:")
print("=" * 80)

risk_scenarios = [
    {'name': 'Conservative (1% risk)', 'win_rate': 0.50, 'avg_win': 0.04, 'avg_loss': 0.02, 'risk': 0.01},
    {'name': 'Moderate (2% risk)', 'win_rate': 0.50, 'avg_win': 0.04, 'avg_loss': 0.02, 'risk': 0.02},
    {'name': 'Aggressive (5% risk)', 'win_rate': 0.50, 'avg_win': 0.04, 'avg_loss': 0.02, 'risk': 0.05},
    {'name': 'Very Aggressive (10% risk)', 'win_rate': 0.50, 'avg_win': 0.04, 'avg_loss': 0.02, 'risk': 0.10},
]

for scenario in risk_scenarios:
    ror = calculate_risk_of_ruin(
        scenario['win_rate'],
        scenario['avg_win'],
        scenario['avg_loss'],
        scenario['risk']
    )
    
    print(f"\n📊 {scenario['name']}:")
    print("-" * 80)
    for key, value in ror.items():
        if isinstance(value, float):
            if '%' in key:
                print(f"  {key:.<50} {value:.4f}%")
            else:
                print(f"  {key:.<50} {value:.4f}")
        else:
            print(f"  {key:.<50} {value}")

print("\n" + "=" * 80)
print("💡 Key Insight: Lower risk per trade = Dramatically lower risk of ruin!")
print("   Even a profitable strategy can ruin you if position sizing is wrong.")

## 5. Exercises

Test your understanding!

---

### Exercise 1: Expected Value Calculation

**Task**: You're evaluating a trading strategy with these statistics:

- Win rate: 45%
- Average winning trade: RM 350
- Average losing trade: RM 150

1. Calculate the expected value per trade
2. Is this strategy profitable?
3. If you take 100 trades, what's your expected total profit/loss?
4. What minimum average win would make this strategy profitable with the same win rate?

---

In [ ]:
# Your code here


### Exercise 2: Position Sizing for Real Trade

**Task**: You want to buy CIMB stock with these parameters:

- Account size: RM 100,000
- Risk tolerance: 1.5% per trade
- Entry price: RM 5.50
- Stop loss: RM 5.20 (technical support level)
- Target price: RM 6.20 (resistance level)

Calculate:
1. How many shares should you buy?
2. Total position value
3. Maximum loss if stopped out
4. Potential profit if target is reached
5. Risk/reward ratio of this trade

---

In [ ]:
# Your code here


### Exercise 3: Kelly Criterion Application

**Task**: You've backtested a trading system with these results over 200 trades:

- Winning trades: 110 (55% win rate)
- Losing trades: 90 (45% loss rate)
- Total profit from wins: RM 22,000 (avg RM 200 per win)
- Total loss from losses: RM 13,500 (avg RM 150 per loss)

Calculate:
1. Full Kelly percentage
2. Half Kelly percentage
3. Quarter Kelly percentage
4. Which would you use and why?
5. If your account is RM 50,000, how much should you risk per trade using Half-Kelly?

---

In [ ]:
# Your code here


### Exercise 4: Drawdown and Recovery Analysis

**Task**: Your account experiences the following monthly returns (in %):

Months 1-12: [5, -3, 8, -2, 10, -15, -8, 12, -5, 7, -4, 6]

Starting capital: RM 50,000

Calculate:
1. Equity curve (account value each month)
2. Maximum drawdown percentage
3. When did the max drawdown occur (which month)?
4. How much gain was needed to recover to the peak?
5. Did the account recover to the peak by month 12?

Create a visualization showing:
- Equity curve
- Running maximum
- Drawdown over time

---

In [ ]:
# Your code here


---

## 📚 Summary

Congratulations! You now understand the **mathematical foundations of risk management**.

### Key Concepts:

1. **Expected Value (EV)**
   - Formula: EV = (P_win × Avg Win) - (P_loss × Avg Loss)
   - **Only trade strategies with positive EV!**
   - Win rate alone doesn't determine profitability

2. **Position Sizing**
   - **Fixed Percentage**: Risk fixed % of capital (most common)
   - **Kelly Criterion**: Optimal bet size for growth
   - **Conservative approach**: Use Half-Kelly or Quarter-Kelly
   - **Never exceed 5% risk** per trade

3. **Maximum Drawdown**
   - Largest peak-to-trough decline
   - Recovery is asymmetric: 50% loss needs 100% gain!
   - **Avoid large drawdowns** - extremely hard to recover

4. **Risk of Ruin**
   - Probability of losing all capital
   - Lower risk per trade → Lower risk of ruin
   - Aim for < 1% risk of ruin

### What You've Learned:

✅ Calculate expected value of trading strategies  
✅ Understand win rate vs risk/reward tradeoffs  
✅ Apply position sizing formulas (fixed %, Kelly)  
✅ Calculate and interpret maximum drawdown  
✅ Assess risk of ruin probability  
✅ Use Monte Carlo simulation for risk analysis  

### 🎯 Practical Rules for Malaysian Traders:

1. **Position Sizing**: Risk 1-2% per trade (conservative)
2. **Stop Losses**: Always use them - no exceptions!
3. **Risk/Reward**: Aim for minimum 1:2 (risk RM1 to make RM2)
4. **Capital Preservation**: Survive first, profit second
5. **Drawdown Limit**: Stop trading if down 20% from peak

### ⚠️ Critical Warnings:

1. **No edge = Guaranteed ruin**: Only trade positive EV strategies
2. **Position sizing > Win rate**: How much you bet matters more than how often you win
3. **One big loss**: Can wipe out months of gains
4. **Emotions**: Math works only if you follow the rules consistently

### 🔢 Essential Formulas:

**Expected Value**:
$$EV = (P_{\text{win}} \times \text{Avg Win}) - (P_{\text{loss}} \times \text{Avg Loss})$$

**Position Sizing**:
$$\text{Shares} = \frac{\text{Account} \times \text{Risk \%}}{\text{Entry Price} - \text{Stop Loss}}$$

**Kelly Criterion**:
$$f^* = \text{Win Rate} - \frac{\text{Loss Rate}}{\text{Win/Loss Ratio}}$$

**Recovery Gain**:
$$\text{Gain Needed} = \frac{\text{Drawdown}}{1 - \text{Drawdown}}$$

---

## 🔜 What's Next?

In **Module 10: Final Project - Complete Stock Analysis**, you'll:
- Choose a Malaysian stock to analyze
- Apply ALL indicators learned (SMA, EMA, RSI, MACD, Bollinger, ATR)
- Calculate risk metrics (correlation, drawdown, position sizing)
- Create a complete trading plan with entry, exit, and risk management
- Present your analysis professionally

This is your **capstone project** - time to put everything together!

**Ready?** Move on to Module 10 when you can:
- ✅ Calculate expected value and determine if a strategy is profitable
- ✅ Size positions using fixed percentage method
- ✅ Understand Kelly Criterion and when to use it
- ✅ Calculate maximum drawdown and recovery needed
- ✅ Complete all exercises without looking at solutions

---

### 📖 Additional Resources:

- [Position Sizing Strategies](https://www.investopedia.com/terms/p/positionsizing.asp)
- [Kelly Criterion Explained](https://www.investopedia.com/articles/trading/04/091504.asp)
- [Maximum Drawdown](https://www.investopedia.com/terms/m/maximum-drawdown-mdd.asp)

---

**Excellent work!** You now have the mathematical tools to manage risk like a professional! 🎉

**Remember**: The best indicator in the world is useless without proper risk management. Master this module and you'll be ahead of 90% of traders!
